In [6]:
import strawberryfields as sf
from strawberryfields.ops import S2gate, Dgate, MeasureX, MeasureP
import numpy as np

def calculate_covariance(X, Y):
    return np.mean(X * Y) - np.mean(X)*np.mean(Y)

def calculate_correlation(X, Y):
    cov = calculate_covariance(X, Y)
    std_X = np.sqrt(np.mean(X**2) - np.mean(X)**2)
    std_Y = np.sqrt(np.mean(Y**2) - np.mean(Y)**2)
    return cov / (std_X * std_Y) if (std_X>0 and std_Y>0) else np.nan

num_runs = 100000
r = 10.0

a = np.random.randint(0, 2, size=num_runs)
b = np.random.randint(0, 2, size=num_runs)
a_adv = np.random.randint(0, 2, size=num_runs)
b_adv = np.random.randint(0, 2, size=num_runs)

M_meas = []
M_prime_meas = []
M_double_prime_meas = []

for i in range(num_runs):
    eng = sf.Engine("gaussian")
    prog = sf.Program(2)

    with prog.context as q:
        S2gate(r) | (q[0], q[1])
        if a[i] == 0:
            MeasureX | q[0]
        else:
            MeasureP | q[0]
        if a_adv[i] == 0:
            MeasureX | q[1]
        else:
            MeasureP | q[1]

    result = eng.run(prog)
    M_val = result.samples[0][0]
    M_prime_val = result.samples[0][1]

    M_meas.append(M_val)
    M_prime_meas.append(M_prime_val)

    eng2 = sf.Engine("gaussian")
    prog2 = sf.Program(1)

    with prog2.context as q2:
        Dgate(M_prime_val, 0 if b_adv[i] == 0 else np.pi/2) | q2[0]
        if b[i] == 0:
            MeasureX | q2[0]
        else:
            MeasureP | q2[0]

    result2 = eng2.run(prog2)
    M_double_prime_val = result2.samples[0][0]

    M_double_prime_meas.append(M_double_prime_val)

M_meas = np.array(M_meas)
M_prime_meas = np.array(M_prime_meas)
M_double_prime_meas = np.array(M_double_prime_meas)

idx_case1 = np.where((a == a_adv) & (b == b_adv))[0]
idx_case2 = np.where((a == a_adv) & (b != b_adv))[0]
idx_case3 = np.where((a != a_adv) & (b == b_adv))[0]
idx_case4 = np.where((a != a_adv) & (b != b_adv))[0]

corr_case1 = calculate_correlation(
    np.abs(M_meas[idx_case1]), 
    np.abs(M_double_prime_meas[idx_case1])
)
corr_case2 = calculate_correlation(
    np.abs(M_meas[idx_case2]), 
    np.abs(M_double_prime_meas[idx_case2])
)
corr_case3 = calculate_correlation(
    np.abs(M_meas[idx_case3]), 
    np.abs(M_double_prime_meas[idx_case3])
)
corr_case4 = calculate_correlation(
    np.abs(M_meas[idx_case4]), 
    np.abs(M_double_prime_meas[idx_case4])
)

print(f"Case 1 (a=a_adv, b=b_adv)         => correlation: {corr_case1:.4f}")
print(f"Case 2 (a=a_adv, b!=b_adv)        => correlation: {corr_case2:.4f}")
print(f"Case 3 (a!=a_adv, b=b_adv)        => correlation: {corr_case3:.4f}")
print(f"Case 4 (a!=a_adv, b!=b_adv)       => correlation: {corr_case4:.4f}")


Case 1 (a=a_adv, b=b_adv)         => correlation: 1.0000
Case 2 (a=a_adv, b!=b_adv)        => correlation: 0.0055
Case 3 (a!=a_adv, b=b_adv)        => correlation: -0.0063
Case 4 (a!=a_adv, b!=b_adv)       => correlation: 0.0189
